# PART 1: Scraping

In [1]:
# Import packages

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time
import pprint
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
###############################################################################################################################

## Scroll through the upcoming races of today and tomorrow with webdriver

##### As a first step, use webdriver to:
##### 1) Scroll through today's races
##### 2) Click on the TOMORROW button
##### 3) Click on the EXPAND ALL button to view all races 
##### 4) Scroll through tomorrow's races

In [3]:
# initialize driver and get website data
website = 'https://www.racingpost.com/racecards/'
driver = webdriver.Chrome()
driver.get(website)

actions = ActionChains(driver)
time.sleep(3)

# scroll to all course images to get an overview of today's upcoming races
course_images = driver.find_elements(By.CLASS_NAME, 'RC-accordion__courseIcon')
for i in course_images:
    actions.move_to_element(i).perform()

# move back to first course image 
actions.move_to_element(course_images[0]).perform()
# Click to the TOMORROW button using XPATH to see tomorrow's races too
#button_xpath = '//*[@id="secondaryNav"]/li[2]/a'
button_xpath = '/html/body/div[2]/div[2]/div/nav/ul/li[2]/ul/li[2]/a'

button = driver.find_element(By.XPATH, button_xpath)
time.sleep(3)
button.click()

time.sleep(3)

# click on the "Expand All" button to see details of all races
expand_button_xpath = "/html/body/div[3]/div[3]/div/main/div[3]/div[2]"
button = driver.find_element(By.XPATH, expand_button_xpath)
time.sleep(3)
button.click()

time.sleep(3)

# scroll through tomorrow's page too
race_arrows = driver.find_elements(By.CLASS_NAME, 'RC-meetingItem__arrow')
for i in race_arrows:
    actions.move_to_element(i).perform()

driver.quit()

In [ ]:
###############################################################################################################################

### Set Selenium with headless option

In [3]:
# Initialize Selenium with headless option for faster execution
chrome_options = Options()
chrome_options.add_argument("--headless")  # Enable headless mode
chrome_options.add_argument("--no-sandbox")  # Recommended to avoid some crashes
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

In [4]:
# Initialize WebDriver with the options
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

In [5]:
# today:
racecard_url = 'https://www.racingpost.com/racecards'

# tomorrow:
#racecard_url = 'https://www.racingpost.com/racecards/tomorrow'

In [6]:
driver.get(racecard_url)

In [7]:
# Get page source and parse it with BeautifulSoup
html_content = driver.page_source
soup = BeautifulSoup(html_content, 'html.parser')

In [8]:
# Extract race URLs from the parsed HTML
def get_race_urls(soup):
    race_urls = []
    for meeting in soup.select('section[data-accordion-row]'):
        course_name = meeting.select_one('span.RC-accordion__courseName').text.strip().lower()
        for race in meeting.select('a.RC-meetingItem__link'):
            race_urls.append('https://www.racingpost.com' + race['href'])
    return sorted(list(set(race_urls)))

race_urls = get_race_urls(soup)
print(str(len(race_urls)) + " upcoming races")
print()
pprint.pprint(str(race_urls[:5]) + "...")

46 upcoming races

("['https://www.racingpost.com/racecards/11/cheltenham/2024-11-15/879054', "
 "'https://www.racingpost.com/racecards/11/cheltenham/2024-11-15/879055', "
 "'https://www.racingpost.com/racecards/11/cheltenham/2024-11-15/879056', "
 "'https://www.racingpost.com/racecards/11/cheltenham/2024-11-15/879057', "
 "'https://www.racingpost.com/racecards/11/cheltenham/2024-11-15/879058']...")


In [9]:
# Limit to the first race URL for testing purposes
#test_race_urls = race_urls[:1]  # Only the first race

In [10]:
#test_race_urls

### Extract Horse Data

In [11]:
# Define a function to extract horse-specific details
def get_horse_details(driver, horse_url):
    driver.get(horse_url)
    try:
        # Wait for the page to load using WebDriverWait
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'hp-nameRow'))
    )
    except TimeoutException:
        print(f"Timeout while loading horse details from {horse_url}")
        return {}  # Return an empty dictionary if the page fails to load in time
    
    horse_soup = BeautifulSoup(driver.page_source, 'html.parser')
    horse_details = {}

    # Extract the country code next to the horse's name
    country_tag = horse_soup.select_one('span.hp-nameRow__code')
    horse_details['country'] = country_tag.text.strip()

    rows = horse_soup.find_all('div', class_='pp-definition')

    # Initialize variables
    dob = None
    trainer_crosscheck = None
    owner = None
    sire = None
    dam = None
    dam_sire = None
    breeder = None
    

    # Loop through the rows and extract the text based on the labels
    for row in rows:
        label = row.find('dt', class_='pp-definition__term').get_text(strip=True)
        content = row.find('dd', class_='pp-definition__description').get_text(strip=True)
        # Clean the content to remove "right"
        content = content.split("right")[0].strip()
    
        if 'yo' in label:
            dob = content
        elif 'Trainer' in label:
            trainer_crosscheck = content
        elif 'Owner' in label:
            owner = content
        elif 'Sire:' == label:
            sire = content
        elif 'Dam:' == label: 
            dam = content
        elif "Dam's Sire:" == label:
            dam_sire = content
        elif 'Breeder' in label: 
            breeder = content
    
    # Store the extracted data in the horse_details dictionary
    horse_details['dob'] = dob
    horse_details['trainer_crosscheck'] = trainer_crosscheck
    horse_details['owner'] = owner
    horse_details['sire'] = sire
    horse_details['dam'] = dam
    horse_details['dam_sire'] = dam_sire
    horse_details['breeder'] = breeder

    
    return horse_details

### Extract Race Data

In [12]:
def get_race_data(driver, url):
    driver.get(url)

    # Wait for the race page to load
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.js-PC-runnerRow'))
    )
    
    # Get the HTML and parse with BeautifulSoup
    race_soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract race details
    race = {}
    race['course'] = race_soup.select_one('h1.RC-courseHeader__name').text.strip()
    race['time'] = race_soup.select_one('span[data-test-selector="RC-courseHeader__time"]').text.strip()
    race['date'] = race_soup.select_one('span[data-test-selector="RC-courseHeader__date"]').text.strip()
    race['race_name'] = race_soup.select_one('span[data-test-selector="RC-header__raceInstanceTitle"]').text.strip()
    race['distance_round'] = race_soup.select_one('strong[data-test-selector="RC-header__raceDistanceRound"]').text.strip()
    race['distance'] = race_soup.select_one('span[data-test-selector="RC-header__raceDistance"]').text.strip() if race_soup.select_one('span[data-test-selector="RC-header__raceDistance"]') else None
    race['age_group'] = race_soup.select_one('span[data-test-selector="RC-header__rpAges"]').text.strip()
    race['class'] = race_soup.select_one('span[data-test-selector="RC-header__raceClass"]').text.strip() if race_soup.select_one('span[data-test-selector="RC-header__raceClass"]') else None

    # Extract the Betting Forecast section
    betting_forecast = {}
    forecast_section = race_soup.find('div', class_='RC-raceFooterInfo_bettingForecast')
    if forecast_section:
        for horse_info in forecast_section.find_all('a', class_='RC-raceFooterInfo__runner'):
            horse_name = horse_info.text.strip()
            odds = horse_info.find_previous('span').text.strip() if horse_info.find_previous('span') else 'N/A'
            odds = odds.split(horse_name)[0].strip().rstrip(',')  # Clean odds text
            
            # Use a regular expression to validate that odds are in the correct format (e.g., '8/1', '10/1')
            if re.fullmatch(r'\d+/\d+', odds):
                betting_forecast[horse_name] = odds
            else:
                # Assign 'N/A' if the odds string is not valid
                betting_forecast[horse_name] = 'N/A'
    
    race['betting_forecast'] = betting_forecast

    # Find all the rows related to the winner, runners, going, etc.
    rows = race_soup.find_all('div', class_='RC-headerBox__infoRow')

    # Initialize variables
    winner = None
    runners_nr = None
    going = None
    stalls = None
    hurdles_nr = None
    fences_nr = None
    ew_terms = None

    # Loop through the rows and extract the text based on the labels
    for row in rows:
        label = row.find('div', class_='RC-headerBox__infoRow__name').get_text(strip=True)
        content = row.find('div', class_='RC-headerBox__infoRow__content').get_text(strip=True)
    
        if 'Winner' in label:
            winner = content
        elif 'Runners' in label:
            runners_nr = content
        elif 'Going' in label:
            going = content
        elif 'Stalls' in label:
            stalls = content
        elif 'No. of hurdles' in label:
            hurdles_nr = content
        elif 'No. of fences' in label:
            fences_nr = content
        elif 'EW Terms' in label:
            ew_terms = content
    
    # Store the extracted data in the race dictionary
    race['winner'] = winner
    race['runners_nr'] = runners_nr
    race['going'] = going
    race['stalls'] = stalls
    race['hurdles_nr'] = hurdles_nr
    race['fences_nr'] = fences_nr
    race['ew_terms'] = ew_terms
    


    # Extract runners
    runners = []
    for runner in race_soup.select('div.js-PC-runnerRow'):
        horse_url = runner.select_one('a[data-test-selector="RC-cardPage-runnerName"]')['href']
        runner_info = {
            'name': runner.select_one('a[data-test-selector="RC-cardPage-runnerName"]').text.strip(),
            'age': runner.select_one('span[data-test-selector="RC-cardPage-runnerAge"]').text.strip(),
            'trainer': runner.select_one('a[data-test-selector="RC-cardPage-runnerTrainer-name"]').text.strip(),
            'jockey': runner.select_one('a[data-test-selector="RC-cardPage-runnerJockey-name"]').text.strip() if runner.select_one('a[data-test-selector="RC-cardPage-runnerJockey-name"]') else None,
            'form': runner.select_one('span[data-test-selector="RC-cardPage-runnerForm"]').text.strip(),
            'runner_nr': runner.select_one('span[data-test-selector="RC-cardPage-runnerNumber-no"]').text.strip(),
            'draw': runner.select_one('span[data-test-selector="RC-cardPage-runnerNumber-draw"]').text.strip(),
            'wgt_st': runner.select_one('span.RC-runnerWgt__carried_st').text.strip(),
            'wgt_lb': runner.select_one('span.RC-runnerWgt__carried_lb').text.strip(),
            'OR': runner.select_one('span[data-test-selector="RC-cardPage-runnerOr"]').text.strip(),
            'allowance': runner.select_one('span[data-test-selector="RC-cardPage-runnerJockey-allowance"]').text.strip(),
            'rtf%': runner.select_one('span[data-test-selector="RC-cardPage-runnerTrainer-rtf"]').text.strip(),
            'TS': runner.select_one('span[data-test-selector="RC-cardPage-runnerTs"]').text.strip(),
            'RPR': runner.select_one('span[data-test-selector="RC-cardPage-runnerRpr"]').text.strip(),
            'horse_url': 'https://www.racingpost.com' + horse_url,  # Full URL of the horse page
        }
        runners.append(runner_info)

    # After extracting the runners, visit each horse page to scrape more details
    for runner in runners:

        horse_name = runner['name'].split("right")[0].strip()
    
        # Add betting forecast to the runner if available
        if horse_name in betting_forecast:
            runner['betting_forecast'] = betting_forecast[horse_name]
        else:
            runner['betting_forecast'] = 'N/A'  # Set to 'N/A' if no forecast is available
        
        # Get additional horse-specific details
        horse_url = runner['horse_url']
        horse_details = get_horse_details(driver, horse_url)  # Get horse-specific details
        runner.update(horse_details)  # Add horse-specific details to runner_info

    race['runners'] = runners
    return race


In [13]:
# Loop through race URLs and extract data
race_data = []
for race_url in race_urls:
    race_data.append(get_race_data(driver, race_url))

In [14]:
# Close the browser
driver.quit() 

In [15]:
# Pretty print first 2 elements of the collected race data
pprint.pprint(race_data[:2])

[{'age_group': '(4yo+)',
  'betting_forecast': {'Boothill': '9/1',
                       'Edwardstone': '7/2',
                       'Jonbon': '4/11',
                       'Unexpected Party': '20/1'},
  'class': '(Class 1)',
  'course': 'Cheltenham',
  'date': '15 Nov 2024                            \n'
          '                    ITV4                \n'
          '\n'
          '\n'
          '\n'
          'help info',
  'distance': '(1m7f199y)',
  'distance_round': '2m',
  'ew_terms': '',
  'fences_nr': '13',
  'going': 'Good',
  'hurdles_nr': None,
  'race_name': 'Shloer Chase (Registered As The Cheltenham Chase) (Grade 2) '
               '(GBB Race)',
  'runners': [{'OR': '164',
               'RPR': '174',
               'TS': '159',
               'age': '10',
               'allowance': '',
               'betting_forecast': '7/2',
               'breeder': 'R Abrey &  I Thurtle',
               'country': '(GB)',
               'dam': 'Nothingtoloose',
               '

# PART 2: Data cleaning + preprocessing

### Convert to DataFrame

In [16]:
# Flatten race data and convert to DataFrame
flattened_data = []
for race in race_data:
    for runner in race['runners']:
        race_runner_data = {
            'course': race['course'],
            'time': race['time'],
            'date': race['date'],
            'race_name': race['race_name'],
            'distance_round': race['distance_round'],
            'distance': race['distance'],
            'age_group': race['age_group'],
            'class': race['class'],
            'winner': race['winner'],
            'runners_nr': race['runners_nr'],
            'going': race['going'],
            'stalls': race['stalls'],
            'hurdles_nr': race['hurdles_nr'],
            'fences_nr': race['fences_nr'],
            'ew_terms': race['ew_terms'],
            'runner_name': runner['name'],
            'age': runner['age'],
            'trainer': runner['trainer'],
            'jockey': runner['jockey'],
            'form': runner['form'],
            'runner_nr': runner['runner_nr'],
            'draw': runner['draw'], 
            'wgt_st': runner['wgt_st'], 
            'wgt_lb': runner['wgt_lb'], 
            'OR': runner['OR'], 
            'allowance': runner['allowance'], 
            'rtf%': runner['rtf%'], 
            'TS': runner['TS'], 
            'RPR': runner['RPR'], 
            'country': runner['country'],
            'dob': runner['dob'],
            'trainer_crosscheck': runner['trainer_crosscheck'],
            'owner': runner['owner'],
            'sire': runner['sire'],
            'dam': runner['dam'],
            'dam_sire': runner['dam_sire'],
            'breeder': runner['breeder'],            
            'betting_forecast': runner['betting_forecast']
        }
        flattened_data.append(race_runner_data)

# Create a DataFrame
df = pd.DataFrame(flattened_data)

In [17]:
# Display the 1st few rows
df.head()

,course,time,date,race_name,distance_round,distance,age_group,class,winner,runners_nr,...,RPR,country,dob,trainer_crosscheck,owner,sire,dam,dam_sire,breeder,betting_forecast
0,Cheltenham,2:20,15 Nov 2024 \n ...,Shloer Chase (Registered As The Cheltenham Cha...,2m,(1m7f199y),(4yo+),(Class 1),"£56,950",4(MAX 20),...,174,(GB),(04Jun14 b g),Alan King,"Robert Abrey, Ian Thurtle",Kayf Tara,Nothingtoloose,Luso,R Abrey & I Thurtle,7/2
1,Cheltenham,2:20,15 Nov 2024 \n ...,Shloer Chase (Registered As The Cheltenham Cha...,2m,(1m7f199y),(4yo+),(Class 1),"£56,950",4(MAX 20),...,176,(FR),(18Apr16 b g),Nicky Henderson,John P McManus,Walk In The Park,Star Face,Saint Des Saints,Lotfi Kohli,4/11
2,Cheltenham,2:20,15 Nov 2024 \n ...,Shloer Chase (Registered As The Cheltenham Cha...,2m,(1m7f199y),(4yo+),(Class 1),"£56,950",4(MAX 20),...,168,(IRE),(11May15 bb g),Harry Fry,Brian & Sandy Lambert,Presenting,Oyster Pipit,Accordion,Noel James,9/1
3,Cheltenham,2:20,15 Nov 2024 \n ...,Shloer Chase (Registered As The Cheltenham Cha...,2m,(1m7f199y),(4yo+),(Class 1),"£56,950",4(MAX 20),...,155,(FR),(06May15 gr g),Dan Skelton,O'Reilly Maclennan Tynan Carthy Shanahan,Martaline,Reform Act,Lemon Drop Kid,Mr & Mrs Gilles Forien,20/1
4,Cheltenham,3:30,15 Nov 2024 \n ...,Albert Bartlett Novices' Hurdle (Grade 2) (Reg...,2m5f,None,(4yo+),(Class 1),"£31,323",6(MAX 18),...,147,(FR),(18Apr19 b g),Dan Skelton,Highclere Thoroughbred Racing - Grand,Bathyrhon,Valaway,Gold Away,"P Rouxel, H Bellentani & G Mousnier",6/4


In [18]:
# Set display options for better visibility 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)           
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 2) 

In [19]:
df.head()

,course,time,date,race_name,distance_round,distance,age_group,class,winner,runners_nr,going,stalls,hurdles_nr,fences_nr,ew_terms,runner_name,age,trainer,jockey,form,runner_nr,draw,wgt_st,wgt_lb,OR,allowance,rtf%,TS,RPR,country,dob,trainer_crosscheck,owner,sire,dam,dam_sire,breeder,betting_forecast
0,Cheltenham,2:20,15 Nov 2024 \n ITV4 \n\n\n\nhelp info,Shloer Chase (Registered As The Cheltenham Chase) (Grade 2) (GBB Race),2m,(1m7f199y),(4yo+),(Class 1),"£56,950",4(MAX 20),Good,None,None,13,,Edwardstone \nright,10,Alan King \nright,Tom Cannon \nright,241F3-,1,,11,10,164,,43,159,174,(GB),(04Jun14 b g),Alan King,"Robert Abrey, Ian Thurtle",Kayf Tara,Nothingtoloose,Luso,R Abrey & I Thurtle,7/2
1,Cheltenham,2:20,15 Nov 2024 \n ITV4 \n\n\n\nhelp info,Shloer Chase (Registered As The Cheltenham Chase) (Grade 2) (GBB Race),2m,(1m7f199y),(4yo+),(Class 1),"£56,950",4(MAX 20),Good,None,None,13,,Jonbon \nright,8,Nicky Henderson \nright,Nico de Boinville \nright,11211-,2,,11,10,170,,32,164,176,(FR),(18Apr16 b g),Nicky Henderson,John P McManus,Walk In The Park,Star Face,Saint Des Saints,Lotfi Kohli,4/11
2,Cheltenham,2:20,15 Nov 2024 \n ITV4 \n\n\n\nhelp info,Shloer Chase (Registered As The Cheltenham Chase) (Grade 2) (GBB Race),2m,(1m7f199y),(4yo+),(Class 1),"£56,950",4(MAX 20),Good,None,None,13,,Boothill \nright,9,Harry Fry \nright,Bryan Carver \nright,11FF4-,3,,11,8,158,,38,157,168,(IRE),(11May15 bb g),Harry Fry,Brian & Sandy Lambert,Presenting,Oyster Pipit,Accordion,Noel James,9/1
3,Cheltenham,2:20,15 Nov 2024 \n ITV4 \n\n\n\nhelp info,Shloer Chase (Registered As The Cheltenham Chase) (Grade 2) (GBB Race),2m,(1m7f199y),(4yo+),(Class 1),"£56,950",4(MAX 20),Good,None,None,13,,Unexpected Party \nright,9,Dan Skelton \nright,Harry Skelton \nright,91P-52,4,,11,8,146,,40,129,155,(FR),(06May15 gr g),Dan Skelton,O'Reilly Maclennan Tynan Carthy Shanahan,Martaline,Reform Act,Lemon Drop Kid,Mr & Mrs Gilles Forien,20/1
4,Cheltenham,3:30,15 Nov 2024 \n ITV4 \n\n\n\nhelp info,Albert Bartlett Novices' Hurdle (Grade 2) (Registered As The Hyde Novices' Hurdle) (GBB Race),2m5f,None,(4yo+),(Class 1),"£31,323",6(MAX 18),Good,None,10,None,,Valgrand \nright,5,Dan Skelton \nright,Harry Skelton \nright,24-111,1,,11,7,140,,40,122,147,(FR),(18Apr19 b g),Dan Skelton,Highclere Thoroughbred Racing - Grand,Bathyrhon,Valaway,Gold Away,"P Rouxel, H Bellentani & G Mousnier",6/4


In [20]:
# Remove unwanted "\nright" from specific columns
df['time'] = df['time'].str.replace(r'\s*\nright', '', regex=True)
df['runner_name'] = df['runner_name'].str.replace(r'\s*\nright', '', regex=True)
df['trainer'] = df['trainer'].str.replace(r'\s*\nright', '', regex=True)
df['jockey'] = df['jockey'].str.replace(r'\s*\nright', '', regex=True)

# Remove superfluous part of date and runners_nr columns
df['date'] = df['date'].str.split("\n").str[0]
df['runners_nr'] = df['runners_nr'].str.split("(").str[0]

# Display the head of the modified columns
df[['time','runner_name','trainer','jockey','date','runners_nr']].head()

,time,runner_name,trainer,jockey,date,runners_nr
0,2:20,Edwardstone,Alan King,Tom Cannon,15 Nov 2024,4
1,2:20,Jonbon,Nicky Henderson,Nico de Boinville,15 Nov 2024,4
2,2:20,Boothill,Harry Fry,Bryan Carver,15 Nov 2024,4
3,2:20,Unexpected Party,Dan Skelton,Harry Skelton,15 Nov 2024,4
4,3:30,Valgrand,Dan Skelton,Harry Skelton,15 Nov 2024,6


In [21]:
# Remove the brackets for specific columns
df['distance'] = df['distance'].str.replace(r'[\(\)]', '', regex=True).str.strip()
df['age_group'] = df['age_group'].str.replace(r'[\(\)]', '', regex=True).str.strip()
df['class'] = df['class'].str.replace(r'[\(\)]', '', regex=True).str.strip()
df['draw'] = df['draw'].str.replace(r'[\(\)]', '', regex=True).str.strip()
df['country'] = df['country'].str.replace(r'[\(\)]', '', regex=True).str.strip()
df['dob'] = df['dob'].str.replace(r'[\(\)]', '', regex=True).str.strip()

# Display the head of the modified columns
df[['distance','age_group','class','draw','country','dob']].head()

,distance,age_group,class,draw,country,dob
0,1m7f199y,4yo+,Class 1,,GB,04Jun14 b g
1,1m7f199y,4yo+,Class 1,,FR,18Apr16 b g
2,1m7f199y,4yo+,Class 1,,IRE,11May15 bb g
3,1m7f199y,4yo+,Class 1,,FR,06May15 gr g
4,None,4yo+,Class 1,,FR,18Apr19 b g


In [22]:
# Split the 'age_group' column into two columns by the whitespace 
split_columns = df['age_group'].str.split(" ", expand=True)

# Assign split columns to 'age_group' and 'rating_band' 
df['age_group'] = split_columns[0]
df['rating_band'] = split_columns[1]

# Verify if it worked as intended
df[['age_group','rating_band']].head(20)

,age_group,rating_band
0,4yo+,None
1,4yo+,None
2,4yo+,None
3,4yo+,None
4,4yo+,None
5,4yo+,None
6,4yo+,None
7,4yo+,None
8,4yo+,None
9,4yo+,None


In [23]:
# View the categories created
print(df['age_group'].unique())
print(df['rating_band'].unique())

['4yo+' '5yo+' '3yo+' '2yo' '3yo' '4-5yo']
[None '0-125' '47-65' '47-70' '0-102' '80-102' '0-130' '0-110' '0-120'
 '0-100' '0-80' '0-90' '0-60' '0-70' '0-75' '0-68' '0-72' '0-65' '0-55'
 '0-115' '0-95']


In [24]:
# Split the dob column into 3 by the spaces
df[['dob','color','gender']] = df['dob'].str.split(expand=True)

# Verify if it worked as intended
df[['dob','color','gender']].head()

,dob,color,gender
0,04Jun14,b,g
1,18Apr16,b,g
2,11May15,bb,g
3,06May15,gr,g
4,18Apr19,b,g


In [25]:
# The currency sign in the winner column gets incorrectly displayed. Drop the sign. 
# Remove the currency sign (£) from the 'winner' column so that it can be converted to numeric later
df['winner'] = df['winner'].str.replace('£', '', regex=False).str.replace(',', '')

# Verify if it worked as intended
df['winner'].head()

0    56950
1    56950
2    56950
3    56950
4    31323
Name: winner, dtype: object

#### Overview

In [26]:
# View a random sample of the DataFrame to have an idea of the values included. 
# Running this multiple times can give us more insight
df.sample(10)

,course,time,date,race_name,distance_round,distance,age_group,class,winner,runners_nr,going,stalls,hurdles_nr,fences_nr,ew_terms,runner_name,age,trainer,jockey,form,runner_nr,draw,wgt_st,wgt_lb,OR,allowance,rtf%,TS,RPR,country,dob,trainer_crosscheck,owner,sire,dam,dam_sire,breeder,betting_forecast,rating_band,color,gender
450,Bahrain,4:35,15 Nov 2024,Bahrain International Trophy (Group 2) (3yo+) (Outer Track) (Turf),1m2f,None,3yo+,None,472441,12,Good,,None,None,,Goemon,5,Haider Ebrahim,Ebrahim Nader,1831-6,3,1,9,2,-,,,-,122,GB,16Mar19,Haider Ebrahim,Mohammed Khalid Abdulrahim,Dark Angel,Spangled,Starspangledbanner,Cheveley Park Stud Limited,50/1,None,gr,h
22,Cheltenham,2:55,15 Nov 2024,Glenfarclas Cross Country Handicap Chase (GBB Race),3m5½f,3m5f56y,5yo+,Class 2,18211,9,Good,None,None,31,,Back On The Lash,10,Martin Keighley,James Best,P08-P0,9,,10,2,140,,44,84,154,GB,21Mar14,Martin Keighley,"Maughan, Redknapp, Ryan, Salters",Malinas,Giovanna,Orpen,Mrs S C Welch & Mr R Phillips,20/1,None,b,g
81,Dundalk (AW),5:15,15 Nov 2024,Christmas Party Nights At Dundalk Stadium Handicap,6f,None,3yo+,None,€7080,14,Standard,,None,None,,Army Of India,6,Ross O'Sullivan,Gary Carroll,70-339,12,7,9,6,72,,63,67,88,IRE,27Mar18,Ross O'Sullivan,Bam Syndicate,Sepoy,Sudfah,Unbridled's Song,Rabbah Bloodstock Limited,8/1,None,gr,g
148,Dundalk (AW),6:15,15 Nov 2024,View Restaurant At Dundalk Stadium Nursery Handicap (Div II),1m,None,2yo,None,€6490,14,Standard,,None,None,,Red Charlie,2,Emmet Mullins,Colin Keane,0706,6,2,9,7,65,,75,-,67,USA,20Mar22,Emmet Mullins,Mrs Margaret O'Rourke,War Of Will,Mutually Benefit,Dynaformer,Glen Hill Farm,9/2,None,b,g
349,Wolverhampton (AW),6:30,15 Nov 2024,Extra Winnings With BetUK Acca Club Handicap,5f,5f21y,3yo+,Class 5,3716,10,Standard,Inside,None,None,,Lipsink,7,Michael Appleby,Alistair Rawlinson,236270,6,4,9,6,69,,41,64,84,IRE,03Mar17,Michael Appleby,William Esdaile,Kodiac,Iron Lips,Iron Mask,Wertheimer Et Frere,N/A,0-72,b,g
292,Abu Dhabi,5:00,15 Nov 2024,Race 6 (Handicap) (Turf),1m3f,None,3yo+,None,8462,10,,,None,None,,El Faras,8,Okbah Aashour,Jose Santiago,-87900,11,10,8,5,54,,,-,-,IRE,18May16,Okbah Aashour,Adelresort,Camelot,Commendable,Pivotal,Joseph Hernon (Ire),N/A,0-80,b,g
65,Dundalk (AW),4:45,15 Nov 2024,Irish Stallion Farms EBF Race,6f,None,2yo,None,€12980,11,Standard,,None,None,,Little Lando,2,David Marnane,Robbie Colgan,0,7,10,9,5,-,,75,15,57,IRE,30Mar22,David Marnane,MRC International,Kodiac,Outschool,Outstrip,Canice Farrell,50/1,None,b,g
27,Cheltenham,4:00,15 Nov 2024,Valda Energy Novices' Handicap Hurdle (GBB Race),2m½f,2m87y,3yo+,Class 3,10075,16,Good,None,8,None,,Balhambar,5,Harry Derham,Paul O'Brien,63112-,5,,11,10,117,,47,113,122,FR,04May19,Harry Derham,Dr Paul Kelly And Vernon Taylor,Almanzor,Moojeh,King's Best,Xavier Moyer & Mme Annabelle Moyer,4/1,0-125,b,g
402,Southwell,12:23,15 Nov 2024,Win £1M With BetMGM's Golden Goals Handicap Chase,3m½f,3m60y,4yo+,Class 4,4700,7,Good,None,None,19,,Mixedwave,7,Pam Sly,Connor Brace,1214-6,5,,11,4,100,,,109,119,IRE,12Apr17,Pam Sly,Pam's People,Gentlewave,Chicklemix,Fair Mix,P M & M H Sly,11/2,0-110,b,g
261,Newcastle,12:45,15 Nov 2024,Hawkstone Stout Available At The Hawk Handicap Hurdle (Go North Brindisi Breeze Series Qualifier),2m7½f,2m7f149y,4yo+,Class 4,4066,10,Good,None,13,None,,Jimli's Island,8,Susan Corbett,Nathan Moscrop,8-1122,7,,10,12,94,,57,51,113,GB,29Apr16,Susan Corbett,Girsonfield Racing Club,Trans Island,Queen Of Diamonds,Fruits Of Love,T C Dawson,7/2,0-110,b,g


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 41 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   course              460 non-null    object
 1   time                460 non-null    object
 2   date                460 non-null    object
 3   race_name           460 non-null    object
 4   distance_round      460 non-null    object
 5   distance            331 non-null    object
 6   age_group           460 non-null    object
 7   class               229 non-null    object
 8   winner              460 non-null    object
 9   runners_nr          460 non-null    object
 10  going               460 non-null    object
 11  stalls              226 non-null    object
 12  hurdles_nr          153 non-null    object
 13  fences_nr           69 non-null     object
 14  ew_terms            460 non-null    object
 15  runner_name         460 non-null    object
 16  age                 460 no

In [28]:
# All datatypes are objects = strings
# Convert variables to appropriate datatypes

# Convert to datetime and then extract only the time component
#df['time'] = pd.to_datetime(df['time'], format='%H:%M', errors='coerce').dt.time
df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.time

# Convert date columns
date_cols = ['date', 'dob']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Convert numeric columns
numeric_cols = ['winner', 'runners_nr', 'hurdles_nr', 'fences_nr', 'age', 'runner_nr', 'draw', 'wgt_st', 'wgt_lb', 'OR', 'allowance', 'rtf%', 'TS',
               'RPR']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Convert categorical columns
cat_cols = ['course','race_name', 'class', 'going','runner_name','trainer', 'jockey', 'country', 'owner','sire', 'dam', 'dam_sire', 'breeder',
            'age_group', 'rating_band', 'color', 'gender']
for col in cat_cols:
    df[col] = df[col].astype('category')

# Convert specific float columns to integer
float_to_int_cols = ['hurdles_nr', 'runner_nr', 'draw', 'allowance', 'TS', 'RPR']

for col in float_to_int_cols:
    # Check if column exists and has values that can be converted
    if col in df.columns:
        # Fill NaN values with 0 or a specific value before conversion, if needed
        df[col] = df[col].fillna(0).astype(int)

# Verify the changes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 41 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   course              460 non-null    category      
 1   time                460 non-null    object        
 2   date                460 non-null    datetime64[ns]
 3   race_name           460 non-null    category      
 4   distance_round      460 non-null    object        
 5   distance            331 non-null    object        
 6   age_group           460 non-null    category      
 7   class               229 non-null    category      
 8   winner              268 non-null    float64       
 9   runners_nr          460 non-null    int64         
 10  going               460 non-null    category      
 11  stalls              226 non-null    object        
 12  hurdles_nr          460 non-null    int32         
 13  fences_nr           69 non-null     float64       

C:\Users\simoe\AppData\Local\Temp\ipykernel_15056\3720881569.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.time
C:\Users\simoe\AppData\Local\Temp\ipykernel_15056\3720881569.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')


In [29]:
# Check if trainer got pulled in correctly from both race page + horse page
df[df['trainer'] != df['trainer_crosscheck']]

,course,time,date,race_name,distance_round,distance,age_group,class,winner,runners_nr,going,stalls,hurdles_nr,fences_nr,ew_terms,runner_name,age,trainer,jockey,form,runner_nr,draw,wgt_st,wgt_lb,OR,allowance,rtf%,TS,RPR,country,dob,trainer_crosscheck,owner,sire,dam,dam_sire,breeder,betting_forecast,rating_band,color,gender


In [30]:
# It did, so the crosscheck column can be dropped
df = df.drop('trainer_crosscheck', axis=1)

# Verify if column got correctly dropped
df.shape

(460, 40)

In [31]:
# Convert distance to meters

# Conversion factors
mile_to_m = 1609.34
furlong_to_m = 201.168
yard_to_m = 0.9144

# Function to convert a distance string to meters
def distance_to_meters(distance_str):
    # If distance is None, return None
    if not distance_str or pd.isna(distance_str):
        return None
    
    # Parse the main distance part (like "2m4½f" or "7f")
    main_distance = re.search(r"(\d+)m", distance_str)
    furlongs = re.search(r"(\d+½?)f", distance_str)
    yards = re.search(r"(\d+)y", distance_str)

    # Calculate meters from miles, furlongs, and yards
    total_meters = 0
    if main_distance:
        total_meters += int(main_distance.group(1)) * mile_to_m
    if furlongs:
        furlong_value = furlongs.group(1).replace("½", ".5")
        total_meters += float(furlong_value) * furlong_to_m
    if yards:
        total_meters += int(yards.group(1)) * yard_to_m

    return round(total_meters, 2)

# Create the 'distance_m' column
df['distance_m'] = df['distance'].apply(lambda x: distance_to_meters(x) if pd.notna(x) else None)

# If 'distance_m' is None, use 'distance_round' instead
df['distance_m'] = df.apply(lambda row: distance_to_meters(row['distance_round']) if pd.isna(row['distance_m']) else row['distance_m'], axis=1)

# Display a random sample of the updated columns
df[['distance', 'distance_round', 'distance_m']].sample(5)

,distance,distance_round,distance_m
255,2m7f149y,2m7½f,4763.10
69,None,6f,1207.01
217,1m7f110y,1m7½f,3118.10
442,1m7f182y,2m,3183.94
246,2m4f51y,2m4f,4069.99


In [32]:
# Convert wgt_st and wgt_lb to weight in kg
df['weight_kg'] = (df['wgt_st'] * 6.35029 + df['wgt_lb'] * 0.453592).round(2)

In [33]:
# Display the updated column heads
df[['wgt_st', 'wgt_lb', 'weight_kg']].head()

,wgt_st,wgt_lb,weight_kg
0,11,10,74.39
1,11,10,74.39
2,11,8,73.48
3,11,8,73.48
4,11,7,73.03


In [34]:
# Display a random sample of the updated columns
df[['wgt_st', 'wgt_lb', 'weight_kg']].sample(5)

,wgt_st,wgt_lb,weight_kg
216,11,2,70.76
452,9,2,58.06
67,9,0,57.15
447,10,9,67.59
288,9,0,57.15


In [35]:
# Check number of columnns
df.shape

(460, 42)

In [36]:
# Drop the old distance and weight columns

df = df.drop(['distance','distance_round','wgt_st','wgt_lb'], axis=1)

In [37]:
# Verify if it worked
df.shape

(460, 38)

In [38]:
# View the column names
df.columns

Index(['course', 'time', 'date', 'race_name', 'age_group', 'class', 'winner',
       'runners_nr', 'going', 'stalls', 'hurdles_nr', 'fences_nr', 'ew_terms',
       'runner_name', 'age', 'trainer', 'jockey', 'form', 'runner_nr', 'draw',
       'OR', 'allowance', 'rtf%', 'TS', 'RPR', 'country', 'dob', 'owner',
       'sire', 'dam', 'dam_sire', 'breeder', 'betting_forecast', 'rating_band',
       'color', 'gender', 'distance_m', 'weight_kg'],
      dtype='object')

In [39]:
# View the number of non-NA cells for each column
df.count()

course              460
time                460
date                460
race_name           460
age_group           460
class               229
winner              268
runners_nr          460
going               460
stalls              226
hurdles_nr          460
fences_nr            69
ew_terms            460
runner_name         460
age                 460
trainer             460
jockey              457
form                460
runner_nr           460
draw                460
OR                  348
allowance           460
rtf%                369
TS                  460
RPR                 460
country             460
dob                 460
owner               460
sire                460
dam                 460
dam_sire            460
breeder             460
betting_forecast    460
rating_band         248
color               460
gender              460
distance_m          460
weight_kg           460
dtype: int64

In [40]:
# View unique course names and their frequency in the dataset
df["course"].value_counts()

course
Dundalk (AW)          98
Punchestown           94
Wolverhampton (AW)    89
Cheltenham            59
Southwell             50
Newcastle             31
Abu Dhabi             27
Bahrain               12
Name: count, dtype: int64

In [41]:
# View unique race names and their frequency in the dataset
df["race_name"].value_counts()

race_name
See You On 23rd & 24th November Handicap Hurdle                                                         21
Lycetts Insurance Brokers Conditional Jockeys Handicap Hurdle (Challenger Stayers Hdl Series Qual')     20
50,000 Euro BetVictor Graduation Bonus Series Maiden Hurdle                                             19
Irishinjuredjockeys.com Maiden                                                                          17
Christmas Party Nights At Dundalk Stadium Handicap                                                      17
Valda Energy Novices' Handicap Hurdle (GBB Race)                                                        16
Race 7 (Handicap) (Turf)                                                                                16
View Restaurant At Dundalk Stadium Nursery Handicap (Div II)                                            14
View Restaurant At Dundalk Stadium Nursery Handicap (Div I)                                             13
Punchestown Premiere Weeken

In [42]:
# View unique horse names and their frequency in the dataset
df["runner_name"].value_counts()

runner_name
Dream Escape          2
Princess Qajar        1
Poyraz Approach       1
Powerful Hook Head    1
Potters Charm         1
Poster Paint          1
Port Noir             1
Port Hedland          1
Poppys On Fire        1
Pompano Beach         1
Polish                1
Point Lonsdale        1
Playtogetaway         1
Plaisir Des Flos      1
Pictures Of Home      1
Phillapa Sue          1
Phantomofthepoints    1
Petit Fusee           1
Pessoa                1
Persian Bliss         1
Party President       1
Park The Giant        1
Our Rosie             1
One More Bottle       1
Officer Of State      1
Not My Problem        1
Not Just Any Eagle    1
Prima Diva            1
Puturhandstogether    1
Neigh Botha           1
Q Twenty Boy          1
Saint Vic             1
Sacred Oath           1
Runaround Sioux       1
Rudy Catrail          1
Rockymountainbleu     1
Rockstown Girl        1
Robindevidastar       1
Risen Again           1
Ricky Langford        1
Revenite              1
Resp

In [43]:
# View unique classes and their frequency in the dataset
df["class"].value_counts()

class
Class 5    84
Class 4    46
Class 3    40
Class 6    36
Class 1    14
Class 2     9
Name: count, dtype: int64

#### Handle null values

In [44]:
# Review incomplete rows
df[df.isnull().any(axis=1)]  

,course,time,date,race_name,age_group,class,winner,runners_nr,going,stalls,hurdles_nr,fences_nr,ew_terms,runner_name,age,trainer,jockey,form,runner_nr,draw,OR,allowance,rtf%,TS,RPR,country,dob,owner,sire,dam,dam_sire,breeder,betting_forecast,rating_band,color,gender,distance_m,weight_kg
0,Cheltenham,02:20:00,2024-11-15,Shloer Chase (Registered As The Cheltenham Chase) (Grade 2) (GBB Race),4yo+,Class 1,56950.0,4,Good,None,0,13.0,,Edwardstone,10,Alan King,Tom Cannon,241F3-,1,0,164.0,0,43.0,159,174,GB,2014-06-04,"Robert Abrey, Ian Thurtle",Kayf Tara,Nothingtoloose,Luso,R Abrey & I Thurtle,7/2,NaN,b,g,3199.48,74.39
1,Cheltenham,02:20:00,2024-11-15,Shloer Chase (Registered As The Cheltenham Chase) (Grade 2) (GBB Race),4yo+,Class 1,56950.0,4,Good,None,0,13.0,,Jonbon,8,Nicky Henderson,Nico de Boinville,11211-,2,0,170.0,0,32.0,164,176,FR,2016-04-18,John P McManus,Walk In The Park,Star Face,Saint Des Saints,Lotfi Kohli,4/11,NaN,b,g,3199.48,74.39
2,Cheltenham,02:20:00,2024-11-15,Shloer Chase (Registered As The Cheltenham Chase) (Grade 2) (GBB Race),4yo+,Class 1,56950.0,4,Good,None,0,13.0,,Boothill,9,Harry Fry,Bryan Carver,11FF4-,3,0,158.0,0,38.0,157,168,IRE,2015-05-11,Brian & Sandy Lambert,Presenting,Oyster Pipit,Accordion,Noel James,9/1,NaN,bb,g,3199.48,73.48
3,Cheltenham,02:20:00,2024-11-15,Shloer Chase (Registered As The Cheltenham Chase) (Grade 2) (GBB Race),4yo+,Class 1,56950.0,4,Good,None,0,13.0,,Unexpected Party,9,Dan Skelton,Harry Skelton,91P-52,4,0,146.0,0,40.0,129,155,FR,2015-05-06,O'Reilly Maclennan Tynan Carthy Shanahan,Martaline,Reform Act,Lemon Drop Kid,Mr & Mrs Gilles Forien,20/1,NaN,gr,g,3199.48,73.48
4,Cheltenham,03:30:00,2024-11-15,Albert Bartlett Novices' Hurdle (Grade 2) (Registered As The Hyde Novices' Hurdle) (GBB Race),4yo+,Class 1,31323.0,6,Good,None,10,NaN,,Valgrand,5,Dan Skelton,Harry Skelton,24-111,1,0,140.0,0,40.0,122,147,FR,2019-04-18,Highclere Thoroughbred Racing - Grand,Bathyrhon,Valaway,Gold Away,"P Rouxel, H Bellentani & G Mousnier",6/4,NaN,b,g,4224.52,73.03
5,Cheltenham,03:30:00,2024-11-15,Albert Bartlett Novices' Hurdle (Grade 2) (Registered As The Hyde Novices' Hurdle) (GBB Race),4yo+,Class 1,31323.0,6,Good,None,10,NaN,,Potters Charm,5,Nigel Twiston-Davies,Sam Twiston-Davies,21-11,2,0,139.0,0,46.0,127,147,IRE,2019-03-28,El Rincon,Valirann,Autumn In New York,Shantou,Gerard P Flynn,5/6,NaN,b,g,4224.52,72.12
6,Cheltenham,03:30:00,2024-11-15,Albert Bartlett Novices' Hurdle (Grade 2) (Registered As The Hyde Novices' Hurdle) (GBB Race),4yo+,Class 1,31323.0,6,Good,None,10,NaN,,Grandad's Cap,4,L J Morgan,Jonjo O'Neill Jr,F-5,3,0,NaN,0,29.0,69,99,IRE,2020-03-05,Alan Rogers,Mahler,Lucy Lamp Light,Luso,Aaron Doocey,100/1,NaN,b,g,4224.52,70.31
7,Cheltenham,03:30:00,2024-11-15,Albert Bartlett Novices' Hurdle (Grade 2) (Registered As The Hyde Novices' Hurdle) (GBB Race),4yo+,Class 1,31323.0,6,Good,None,10,NaN,,Wildfell,4,D J Jeffreys,Kielan Woods,,4,0,NaN,0,38.0,0,0,IRE,2020-04-12,D J Jeffreys,Tasleet,Indian Ink,Indian Ridge,Ballyhane,N/A,NaN,ch,g,4224.52,70.31
8,Cheltenham,03:30:00,2024-11-15,Albert Bartlett Novices' Hurdle (Grade 2) (Registered As The Hyde Novices' Hurdle) (GBB Race),4yo+,Class 1,31323.0,6,Good,None,10,NaN,,Gale Mahler,5,Adrian Paul Keatley,Henry Brooke,111112,5,0,133.0,0,NaN,135,145,IRE,2019-05-28,J Fyffe,Mahler,Victrix Gale,Presenting,A J Martin,7/1,NaN,b,m,4224.52,69.85
9,Cheltenham,03:30:00,2024-11-15,Albert Bartlett Novices' Hurdle (Grade 2) (Registered As The Hyde Novices' Hurdle) (GBB Race),4yo+,Class 1,31323.0,6,Good,None,10,NaN,,Rockstown Girl,6,Anthony McCann,James O'Sullivan,6-1431,6,0,122.0,0,50.0,127,135,IRE,2018-01-26,Rockstown Girl Syndicate,Iffraaj,Miss Marina Bay,Galileo,Tinnakill Bloodstock,16/1,NaN,ch,m,4224.52,67.59


In [45]:
# Display a summary of missing values in each column
df.isnull().sum()

course                0
time                  0
date                  0
race_name             0
age_group             0
class               231
winner              192
runners_nr            0
going                 0
stalls              234
hurdles_nr            0
fences_nr           391
ew_terms              0
runner_name           0
age                   0
trainer               0
jockey                3
form                  0
runner_nr             0
draw                  0
OR                  112
allowance             0
rtf%                 91
TS                    0
RPR                   0
country               0
dob                   0
owner                 0
sire                  0
dam                   0
dam_sire              0
breeder               0
betting_forecast      0
rating_band         212
color                 0
gender                0
distance_m            0
weight_kg             0
dtype: int64

In [46]:
# Strip whitespace and replace empty strings with NaN
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)  # Remove leading/trailing whitespace
df.replace('', np.nan, inplace=True)  # Replace empty strings with NaN

# Define additional placeholders to consider as missing values
placeholders = ["NaN", "None", "N/A", "-"]

# Replace placeholder values with np.nan so they are counted as missing
df_cleaned = df.replace(placeholders, np.nan)

# Check for missing values in each column, including the placeholders and empty strings
missing_values_summary = df_cleaned.isnull().sum()

# Display the summary of missing values
print(missing_values_summary)

course                0
time                  0
date                  0
race_name             0
age_group             0
class               231
winner              192
runners_nr            0
going                27
stalls              371
hurdles_nr            0
fences_nr           391
ew_terms            460
runner_name           0
age                   0
trainer               0
jockey                3
form                 26
runner_nr             0
draw                  0
OR                  112
allowance             0
rtf%                 91
TS                    0
RPR                   0
country               0
dob                   0
owner                 0
sire                  0
dam                   0
dam_sire              0
breeder               0
betting_forecast    134
rating_band         212
color                 0
gender                0
distance_m            0
weight_kg             0
dtype: int64


In [47]:
# Display a summary with percentages
missing_percentage = (df_cleaned.isnull().mean() * 100).round(2)

# Combine counts and percentages in a summary DataFrame
missing_summary = pd.DataFrame({
    'Missing Values': df_cleaned.isnull().sum(),
    'Percentage': missing_percentage
})

# Filter to show only columns with missing values
missing_summary = missing_summary[missing_summary['Missing Values'] > 0]

print("Summary of Missing Values per Column:")
print(missing_summary)

Summary of Missing Values per Column:
                  Missing Values  Percentage
class                        231       50.22
winner                       192       41.74
going                         27        5.87
stalls                       371       80.65
fences_nr                    391       85.00
ew_terms                     460      100.00
jockey                         3        0.65
form                          26        5.65
OR                           112       24.35
rtf%                          91       19.78
betting_forecast             134       29.13
rating_band                  212       46.09


In [48]:
# Drop columns with high percentages of missing data (60+%)
df = df.drop(['stalls','fences_nr','ew_terms'], axis=1)

In [49]:
# Verify if it worked
df.columns

Index(['course', 'time', 'date', 'race_name', 'age_group', 'class', 'winner',
       'runners_nr', 'going', 'hurdles_nr', 'runner_name', 'age', 'trainer',
       'jockey', 'form', 'runner_nr', 'draw', 'OR', 'allowance', 'rtf%', 'TS',
       'RPR', 'country', 'dob', 'owner', 'sire', 'dam', 'dam_sire', 'breeder',
       'betting_forecast', 'rating_band', 'color', 'gender', 'distance_m',
       'weight_kg'],
      dtype='object')

In [50]:
# View the number of columns left
df.shape

(460, 35)

In [51]:
# In the categorical columns, fill missing values with "Unknown" for classification purposes
df['class'] = df['class'].cat.add_categories("Unknown").fillna("Unknown")
df['jockey'] = df['jockey'].cat.add_categories("Unknown").fillna("Unknown")
df['rating_band'] = df['rating_band'].cat.add_categories("Unknown").fillna("Unknown")
df['form'] = df['form'].fillna("Unknown")
df['betting_forecast'] = df['betting_forecast'].fillna("Unknown")

In [52]:
# Impute missing values in specific numerical columns with the median value of the column
or_median = df['OR'].median()
df['OR'] = df['OR'].fillna(or_median)

rtf_median = df['rtf%'].median()
df['rtf%'] = df['rtf%'].fillna(rtf_median)

In [53]:
# Impute missing values in the winner column using the mean of races with similar values in the class and distance_m columns

# Step 1: Create distance bins of 100 meters
df['distance_bin'] = pd.cut(df['distance_m'], bins=range(0, int(df['distance_m'].max()) + 100, 100))

# Step 2: Group by 'class' and 'distance_bin', then calculate the mean of 'winner' within each group
winner_mean_class_distance_bin = df.groupby(['class', 'distance_bin'])['winner'].transform('mean')

# Step 3: Impute missing 'winner' values based on 'class' and 'distance_bin'
df['winner'] = df['winner'].fillna(winner_mean_class_distance_bin)

# Display the updated DataFrame to check if missing values in 'winner' have been filled
df[['class', 'distance_bin', 'winner']].head()

C:\Users\simoe\AppData\Local\Temp\ipykernel_15056\1941976420.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  winner_mean_class_distance_bin = df.groupby(['class', 'distance_bin'])['winner'].transform('mean')


,class,distance_bin,winner
0,Class 1,"(3100, 3200]",56950.0
1,Class 1,"(3100, 3200]",56950.0
2,Class 1,"(3100, 3200]",56950.0
3,Class 1,"(3100, 3200]",56950.0
4,Class 1,"(4200, 4300]",31323.0


In [54]:
df['winner'].unique()

array([ 56950.,  31323.,  22780.,  18211.,  10075.,   9803.,     nan,
         8462.,   6179.,   4066.,   3248.,   2451.,   4711.,   3716.,
         3402.,   2983.,   4700.,   4193., 472441.])

In [55]:
# View if there are any missing values left 
df['winner'].isna().sum()

165

In [56]:
# For the ones that are still missing impute based on 'class' alone:
winner_mean_class = df.groupby('class')['winner'].transform('mean')
df['winner'] = df['winner'].fillna(winner_mean_class)

C:\Users\simoe\AppData\Local\Temp\ipykernel_15056\2745188776.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  winner_mean_class = df.groupby('class')['winner'].transform('mean')


In [57]:
df['winner'].isna().sum()

0

In [58]:
# Drop the 'distance_bin' column after imputation, as it is no longer needed
df = df.drop(columns=['distance_bin'])

#### Check for dupliactes

In [59]:
df[df.duplicated()]

,course,time,date,race_name,age_group,class,winner,runners_nr,going,hurdles_nr,runner_name,age,trainer,jockey,form,runner_nr,draw,OR,allowance,rtf%,TS,RPR,country,dob,owner,sire,dam,dam_sire,breeder,betting_forecast,rating_band,color,gender,distance_m,weight_kg


In [60]:
# Drop duplicates if there are any in a new dataframe that is duplicate-free
df_dup_free = df.drop_duplicates()

# Check if it worked
df_dup_free.shape

(460, 35)

In [61]:
# (The above is for demonstration purposes, it is highly unlikely to have any duplicates in this particular dataset)

####  Check if values lie in the expected range

In [62]:
# View summary statistics of numeric columns
df.describe()

,date,winner,runners_nr,hurdles_nr,age,runner_nr,draw,OR,allowance,rtf%,TS,RPR,dob,distance_m,weight_kg
count,460,460.00,460.00,460.00,460.00,460.00,460.00,460.00,460.00,460.00,460.00,460.00,460,460.00,460.00
mean,2024-11-15 00:00:00,50221.70,11.40,3.31,5.08,6.10,3.22,85.14,1.84,50.11,46.33,80.32,2019-03-04 13:49:33.913043712,2842.76,65.58
min,2024-11-15 00:00:00,2451.00,2.00,0.00,2.00,0.00,0.00,46.00,0.00,10.00,0.00,0.00,2011-05-11 00:00:00,1025.04,53.07
25%,2024-11-15 00:00:00,4066.00,9.00,0.00,3.00,3.00,0.00,70.00,0.00,39.50,0.00,68.00,2017-05-25 12:00:00,1609.34,59.76
50%,2024-11-15 00:00:00,9803.00,11.00,0.00,5.00,5.00,0.00,82.50,0.00,50.00,52.00,85.00,2019-04-08 00:00:00,3118.10,64.41
75%,2024-11-15 00:00:00,92821.82,14.00,8.00,7.00,9.00,6.00,96.00,3.00,60.00,79.00,112.25,2021-03-02 12:00:00,4069.99,72.12
max,2024-11-15 00:00:00,472441.00,21.00,13.00,13.00,21.00,17.00,170.00,10.00,100.00,164.00,176.00,2022-05-17 00:00:00,5885.07,78.02
std,NaN,80488.48,4.24,4.80,2.24,4.34,4.27,21.98,2.91,18.22,41.66,43.08,NaN,1265.24,6.75


In [63]:
# Max age is way above standard deviation from mean
# View location where age is max

df[df['age'] == df['age'].max()]

,course,time,date,race_name,age_group,class,winner,runners_nr,going,hurdles_nr,runner_name,age,trainer,jockey,form,runner_nr,draw,OR,allowance,rtf%,TS,RPR,country,dob,owner,sire,dam,dam_sire,breeder,betting_forecast,rating_band,color,gender,distance_m,weight_kg
168,Punchestown,12:15:00,2024-11-15,Punchestown Premiere Weekend Is Next Handicap Chase,4yo+,Unknown,92821.82,13,Good,0,Rudy Catrail,13,E Sheehy,Harry Sexton,P-8663,12,0,72.0,7,50.0,70,96,IRE,2011-05-11,Well Horse Syndicate,Nazar,Rudy Susan,Rudimentary,Jeremiah Aherne,N/A,0-102,b,g,4892.03,63.5


In [64]:
# Max winner is 4x as much as Q3

df[df['winner'] == df['winner'].max()]

,course,time,date,race_name,age_group,class,winner,runners_nr,going,hurdles_nr,runner_name,age,trainer,jockey,form,runner_nr,draw,OR,allowance,rtf%,TS,RPR,country,dob,owner,sire,dam,dam_sire,breeder,betting_forecast,rating_band,color,gender,distance_m,weight_kg
448,Bahrain,04:35:00,2024-11-15,Bahrain International Trophy (Group 2) (3yo+) (Outer Track) (Turf),3yo+,Unknown,472441.0,12,Good,0,Alflaila,5,Owen Burrows,Jim Crowley,15-410,1,10,82.5,0,50.0,0,132,GB,2019-04-02,Shadwell Estate Company Ltd,Dark Angel,Adhwaa,Oasis Dream,Shadwell Estate Company Limited,9/2,Unknown,b,h,2011.68,58.06
449,Bahrain,04:35:00,2024-11-15,Bahrain International Trophy (Group 2) (3yo+) (Outer Track) (Turf),3yo+,Unknown,472441.0,12,Good,0,Calif,5,C & Y Lerner,Adrie de Vries,-37114,2,6,82.5,0,50.0,0,132,GER,2019-04-19,Victorious Racing,Areion,Cherry Danon,Rock Of Gibraltar,Gestut Brummerhof,12/1,Unknown,bb,g,2011.68,58.06
450,Bahrain,04:35:00,2024-11-15,Bahrain International Trophy (Group 2) (3yo+) (Outer Track) (Turf),3yo+,Unknown,472441.0,12,Good,0,Goemon,5,Haider Ebrahim,Ebrahim Nader,1831-6,3,1,82.5,0,50.0,0,122,GB,2019-03-16,Mohammed Khalid Abdulrahim,Dark Angel,Spangled,Starspangledbanner,Cheveley Park Stud Limited,50/1,Unknown,gr,h,2011.68,58.06
451,Bahrain,04:35:00,2024-11-15,Bahrain International Trophy (Group 2) (3yo+) (Outer Track) (Turf),3yo+,Unknown,472441.0,12,Good,0,Killer Ability,5,Takashi Saito,Oisin Murphy,04-200,4,9,82.5,0,50.0,0,126,JPN,2019-01-27,Carrot Farm Co Ltd,Deep Impact,Killer Graces,Congaree,Northern Racing,16/1,Unknown,bb,h,2011.68,58.06
452,Bahrain,04:35:00,2024-11-15,Bahrain International Trophy (Group 2) (3yo+) (Outer Track) (Turf),3yo+,Unknown,472441.0,12,Good,0,Nations Pride,5,Charlie Appleby,William Buick,17-321,5,8,82.5,0,40.0,0,134,IRE,2019-04-28,Godolphin,Teofilo,Important Time,Oasis Dream,Godolphin,N/A,Unknown,b,h,2011.68,58.06
453,Bahrain,04:35:00,2024-11-15,Bahrain International Trophy (Group 2) (3yo+) (Outer Track) (Turf),3yo+,Unknown,472441.0,12,Good,0,Point Lonsdale,5,A P O'Brien,Ryan Moore,061353,6,5,82.5,0,69.0,0,127,IRE,2019-03-01,"D Smith, Mrs J Magnier, M Tabor, Westerberg",Australia,Sweepstake,Acclamation,Epona Bloodstock Ltd,N/A,Unknown,b,h,2011.68,58.06
454,Bahrain,04:35:00,2024-11-15,Bahrain International Trophy (Group 2) (3yo+) (Outer Track) (Turf),3yo+,Unknown,472441.0,12,Good,0,Sovereign Spirit,4,Jaber Ramadhan,Soufiane Saadi,274121,7,12,82.5,0,50.0,0,121,GB,2020-04-28,Al-Afoo Racing Stable,Le Havre,Shutka,Holy Roman Emperor,J P Dubois,33/1,Unknown,b,g,2011.68,58.06
455,Bahrain,04:35:00,2024-11-15,Bahrain International Trophy (Group 2) (3yo+) (Outer Track) (Turf),3yo+,Unknown,472441.0,12,Good,0,Spirit Dancer,7,Richard Fahey,Oisin Orr,1-4107,8,11,82.5,0,62.0,0,129,GB,2017-03-14,Done Ferguson Mason,Frankel,Queen's Dream,Oasis Dream,Sir Alex Ferguson & Mr Niall McLoughlin (Gb),N/A,Unknown,b,g,2011.68,58.06
456,Bahrain,04:35:00,2024-11-15,Bahrain International Trophy (Group 2) (3yo+) (Outer Track) (Turf),3yo+,Unknown,472441.0,12,Good,0,Yamanin Sympa,6,Takashi Saito,Taisei Danno,33-498,9,4,82.5,0,50.0,0,123,JPN,2018-05-04,H Doi,Deep Impact,Yamaninpapillonner,Swept Overboard,Nishikioka Farm,14/1,Unknown,gr,h,2011.68,58.06
457,Bahrain,04:35:00,2024-11-15,Bahrain International Trophy (Group 2) (3yo+) (Outer Track) (Turf),3yo+,Unknown,472441.0,12,Good,0,Young Ireland,4,Jaber Ramadhan,David Liska,27-526,10,7,82.5,0,50.0,0,106,IRE,2020-02-11,Al-Afoo Racing Stable,New Approach,Ard Fheis,Lil's Boy,J S Bolger,66/1,Unknown,br,g,2011.68,58.06


In [73]:
# Define expected ranges
expected_ranges = {
    'winner': (2000, 100000),  
    'runners_nr': (2, 20),
    'age': (2, 12),
    'draw': (0, 20),
    'weight_kg': (50, 80),
    'OR': (0, 150),
    'rtf%': (0, 100),
    'distance_m': (1000, 6000)
}

# Check if each column lies within the expected range and print any out-of-range values
for column, (min_val, max_val) in expected_ranges.items():
    out_of_range = df[(df[column] < min_val) | (df[column] > max_val)]
    if not out_of_range.empty:
        print(f"Out-of-range values for {column}:")
        print(out_of_range[[column]])

Out-of-range values for winner:
       winner
448  472441.0
449  472441.0
450  472441.0
451  472441.0
452  472441.0
453  472441.0
454  472441.0
455  472441.0
456  472441.0
457  472441.0
458  472441.0
459  472441.0
Out-of-range values for runners_nr:
     runners_nr
230          21
231          21
232          21
233          21
234          21
235          21
236          21
237          21
238          21
239          21
240          21
241          21
242          21
243          21
244          21
245          21
246          21
247          21
248          21
249          21
250          21
Out-of-range values for age:
     age
168   13
Out-of-range values for OR:
       OR
0   164.0
1   170.0
2   158.0
12  154.0
14  166.0


#### Identify and handle outliers 

In [66]:
# Identify outliers using IQR (= interquartile range)
outlier_cols = ['winner', 'runners_nr', 'hurdles_nr', 'age', 'OR', 'distance_m', 'weight_kg'] 
for col in outlier_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Find and print outliers
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    if not outliers.empty:
        print(f"Outliers detected in {col}:")
        print(outliers[[col]])

Outliers detected in winner:
       winner
448  472441.0
449  472441.0
450  472441.0
451  472441.0
452  472441.0
453  472441.0
454  472441.0
455  472441.0
456  472441.0
457  472441.0
458  472441.0
459  472441.0
Outliers detected in OR:
       OR
0   164.0
1   170.0
2   158.0
3   146.0
4   140.0
5   139.0
10  145.0
11  144.0
12  154.0
13  138.0
14  166.0
15  143.0
16  143.0
17  140.0
18  140.0
19  140.0
20  140.0
21  140.0
22  140.0


The theoretical maximum of OR is 200.

In [67]:
# View outliers in OR

df[df["OR"] > 200]

,course,time,date,race_name,age_group,class,winner,runners_nr,going,hurdles_nr,runner_name,age,trainer,jockey,form,runner_nr,draw,OR,allowance,rtf%,TS,RPR,country,dob,owner,sire,dam,dam_sire,breeder,betting_forecast,rating_band,color,gender,distance_m,weight_kg


In [68]:
# Cap OR values at 200
df['OR'] = df['OR'].clip(upper=200)

# Check if the values have been capped
print(df['OR'].describe())

count    460.00
mean      85.14
std       21.98
min       46.00
25%       70.00
50%       82.50
75%       96.00
max      170.00
Name: OR, dtype: float64


In [69]:
# Weight in kg must be between 50 kg and 80 kg
# View the outliers
df[(df["weight_kg"] < 50) | (df["weight_kg"] > 80)]

,course,time,date,race_name,age_group,class,winner,runners_nr,going,hurdles_nr,runner_name,age,trainer,jockey,form,runner_nr,draw,OR,allowance,rtf%,TS,RPR,country,dob,owner,sire,dam,dam_sire,breeder,betting_forecast,rating_band,color,gender,distance_m,weight_kg


In [70]:
# Cap the values accordingly 
df['weight_kg'] = df['weight_kg'].clip(lower=50, upper=80)

# Check if the values have been capped
print(df['weight_kg'].describe())

count    460.00
mean      65.58
std        6.75
min       53.07
25%       59.76
50%       64.41
75%       72.12
max       78.02
Name: weight_kg, dtype: float64


In [71]:
# # Export to CSV
df.to_csv('horse_racing_data_today.csv', index=False)
#df.to_csv('horse_racing_data_tomorrow.csv', index=False)

In [74]:
###########################################################################################################################

### Merging 

##### Only run this part after both today's and tomorrow's datasets got exported

In [75]:
# Load today's and tomorrow's datasets
today_df = pd.read_csv('horse_racing_data_today.csv')
tomorrow_df = pd.read_csv('horse_racing_data_tomorrow.csv')

In [76]:
# View dimensions of DataFrames
print(today_df.shape)
print(tomorrow_df.shape)

(460, 35)
(555, 35)


In [78]:
# Combine the datasets
combined_df = pd.concat([today_df, tomorrow_df], ignore_index=True)

# View dimensions of combined DataFrame
print(combined_df.shape)

(1015, 35)


In [79]:
# Check for duplicates
combined_df[combined_df.duplicated()]

,course,time,date,race_name,age_group,class,winner,runners_nr,going,hurdles_nr,runner_name,age,trainer,jockey,form,runner_nr,draw,OR,allowance,rtf%,TS,RPR,country,dob,owner,sire,dam,dam_sire,breeder,betting_forecast,rating_band,color,gender,distance_m,weight_kg


In [80]:
# Drop duplicate entries if there are any
combined_df = combined_df.drop_duplicates()

In [82]:
# Export the combined dataset
combined_df.to_csv('upcoming_races.csv', index=False)